In [ ]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [ ]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier, early_stopping
from sklearn.svm import SVC


In [ ]:
# upload training and test set
X_train = pd.read_csv('data/classification/DCR/X_train.csv', index_col=0)
X_test = pd.read_csv('data/classification/DCR/X_val.csv', index_col=0)
y_train = pd.read_csv('data/classification/DCR/y_train.csv', index_col=0)
y_test = pd.read_csv('data/classification/DCR/y_val.csv', index_col=0)
y_ext = pd.read_csv('data/classification/DCR/y_test.csv', index_col=0)

y_train=y_train.squeeze()
y_test=y_test.squeeze()

In [ ]:
# import functions from the file "functions.ipynb" to perform feature selection
from ipynb.fs.full.functions import MRMR
from ipynb.fs.full.functions import recursive_feature_elimination, recursive_feature_elimination_MLP
from ipynb.fs.full.functions import fw_bw_feature_selection

In [ ]:
names = ['XGB', 'LR', 'RF', 'MLP','SVM','AB','ET','LGBM']

In [ ]:
# selection is performed with default hyperparameters
classifiers = [
          XGBClassifier(),
          LogisticRegression(max_iter = 200), 
          RandomForestClassifier(),
          MLPClassifier(),
          SVC(probability = True), 
          AdaBoostClassifier(),
          ExtraTreesClassifier(),
          LGBMClassifier()
            ]

## MRMR

In [ ]:
selected_features_MRMR = []


for i in range(0, len(classifiers)):
    sel_feats_MRMR = MRMR(X_train, y_train, classifiers[i])
    print('Selecting on {m}'.format(m = names[i]))
    selected_features_MRMR.append(sel_feats_MRMR)
    print('----------------------------------------------')
    print('----------------------------------------------')


In [ ]:
# save the selected features into CSV file
pd.DataFrame(selected_features_MRMR).to_csv('MRMRfeats.csv')

## RFE

In [ ]:
selected_features_RFE = []

for i in range(0, len(classifiers)):
    if(i!=3):
        sel_feats_RFE = recursive_feature_elimination(X_train,y_train,classifiers[i])
        print('Selecting on {m}'.format(m = names[i]))
        selected_features_RFE.append(sel_feats_RFE)
        print('----------------------------------------------')
        print('----------------------------------------------')

In [ ]:
# perform RFE for MLP because it doesn't have feature importance and its needed permutation importance with eli5
RFE_MLP = recursive_feature_elimination_MLP(X_tr, y_train, classifiers[3])
selected_features_RFE.insert(3,RFE_MLP)

In [ ]:
pd.DataFrame(selected_features_RFE).to_csv('RFEfeats.csv')


## FFS

In [ ]:
selected_features_FFS = []

for i in range(0, len(classifiers)):
    print('Selecting on {m}'.format(m = names[i]))
    sel_feats_FFS = fw_bw_feature_selection(X_train,y_train,classifiers[i])
    selected_features_FFS.append(sel_feats_FFS)
    print('----------------------------------------------')
    print('----------------------------------------------')
    

In [ ]:
pd.DataFrame(selected_features_FFS).to_csv('FFSfeats.csv')

## BFS

In [ ]:
selected_features_BFS = []

for i in range(0, len(classifiers)):
    print('Selecting on {m}'.format(m = names[i]))
    sel_feats_BFS = fw_bw_feature_selection(X_train,y_train,classifiers[i],'backward')
    selected_features_BFS.append(sel_feats_BFS)
    print('----------------------------------------------')
    print('----------------------------------------------')
    

In [ ]:
pd.DataFrame(selected_features_BFS).to_csv('BFSfeats.csv')